In [1]:
import pandas as pd
from string import punctuation
import numpy as np
import multiprocessing
import tqdm

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

import rutermextract
term_extractor = rutermextract.TermExtractor()

from pymystem3 import Mystem
mystem = Mystem() 

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/royalcat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
questions_models = {
    "Кладовщик":{
        "q1": "Желаемая должность",
        "q2": "У вас есть опыт работы кладовщиком?",
        "q2d": "У вас есть опыт работы кладовщиком? строка",
        "q3": "Укажите Ваш стаж работы кладовщиком",
        "q4": "Отрасль",
        "q5": "Названия компаний",
        "q6": "С какой системой хранения Вы имели опыт работы?",
        "q7": "Опыт работы с программами",
        "q8": "Опыт с инструментарием",
        "q9": "Типы работ",
        "q10": "Сколько времени вы потратили на поиск работы кладовщика?",
        "q11": "Уровень ЗП (желаемый)"
    },
    "Водитель погрузчика":{
        "q1": "Желаемая должность",
        "q2": "У вас есть водительские права?",
        "q3": "Водительское удостоверение",
        "q4": "Удостоверение тракториста-машиниста нового образца",
        "q5": "Удостоверение тракториста-машиниста старого образца",
        "q6": "У вас есть опыт работы водителем погрузчиком?",
        "q7": "Ваш стаж работы водителем погрузчика",
        "q8": "Сфера деятельности",
        "q9": "Названия компаний",
        "q10": "Какой техникой вы управляете?",
        "q11": "Максимальная высота поднятия груза",
        "q12": "С какой системой хранения Вы имели опыт работы?",
        "q13": "удостоверение стропальщика?",
        "q14": "Типы работ",
        "q15": "Возможность провести тест-драйв на собеседовании",
        "q16": "Ключевые качества",
        "q17": "Знание языков",
        "q19": "Хобби",
        "q20": "Уровень ЗП",
        "q21": "График работы",
        "q22": "Ночная смена",
        "q23": "Командировки",
        "q24": "Станции метро"
    },
    "Официант":{
        "q1": "Желаемая должность",
        "q2": "Есть ли у Вас профессиональное образование в сфере гостеприимства?",
        "q3": "профессиональное образовательное учреждение, где вы учились, и специальность",
        "q4": "Год окончания обучения",
        "q5": "Стаж работы официантом",
        "q6": "Рестораны, где работал",
        "q7": "дополнительные профессиональные знания и навыки",
        "q8": "Знание языков",
        "q9": "C какой посудой работали",
        "q10": "График работы"
    }
}

In [3]:
df_resume = pd.read_csv('./../resume_to_hackaton.csv', sep=';')
df_vac = pd.read_csv('./../vacancies_to_hackaton.csv', sep=';')
target_positions = list(questions_models.keys())

In [4]:
df_target_vac = df_vac[df_vac['name'].isin(target_positions)]
df_target_resume = df_resume[df_resume['position'].isin(target_positions)]

In [83]:
df_target_resume['description'].sample(1).values

'Встреча и приветствие гостей\r  Обслуживание гостей кофейни\r  Консультирование гостей по ассортименту кофейни\r  Контроль чистоты кофейни: поддержание порядка в зале, подсобных помещениях'

In [21]:
print(df_target_vac.iloc[5]['name'])
print(df_target_vac.iloc[5]['description'])

Кладовщик
Обязанности:  Осуществлять работу по приему, хранению и отпуску товарно-материальных ценностей, по их размещению с учетом наиболее рационального использования складских площадей, облегчения и ускорения поиска необходимых материалов, инвентаря и т.п.; Обеспечивать сохранность складируемых товарно-материальных ценностей; Соблюдать правила и порядок хранения и складирования товарно-материальных ценностей; Участвовать в проведении инвентаризаций товарно-материальных ценностей;  Требования:  Опыт работы в должности кладовщика; Знание правил эксплуатации средств вычислительной техники, коммуникаций и связи; Знание складского документооборота; Опыт работы в складских ИТ-системах, знание Excel; Уверенный пользователь ПК.  Условия:  Оформление согласно ТК РФ; 5-ти дневная рабочая неделя; Работа в теплом складе на территории Полиграфической компании "Конфлекс-СПБ". 


In [5]:
def preprocess_text(text: str, word_limit: int):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token.split(" ") for token in tokens]
    tokens = np.concatenate(tokens)
    tokens = [token.strip() for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    text = " ".join(tokens)
    terms = term_extractor(text, limit=word_limit, strings=True)
    
    return terms

In [6]:
def process_text_data(question: str, text: str):
    question_terms = preprocess_text(question, 2) 
    text_terms = preprocess_text(text, 20)
    return question_terms, text_terms

In [7]:
train_df = pd.DataFrame()
train_data = []
for i, row in df_target_vac.iterrows():
    position = row['name']
    text = row['description']
    if position in questions_models:
        questions_model = questions_models[position]
        for question in list(questions_model.values())[1:]:
            train_data.append({
                "position": position,
                "question": question,
                "text": text
            })

In [8]:
train_df = pd.DataFrame(train_data)
train_df

,position,question,text
0,Официант,Есть ли у Вас профессиональное образование в с...,"Мы ищем активного, жизнерадостного и дружелюбн..."
1,Официант,"профессиональное образовательное учреждение, г...","Мы ищем активного, жизнерадостного и дружелюбн..."
2,Официант,Год окончания обучения,"Мы ищем активного, жизнерадостного и дружелюбн..."
3,Официант,Стаж работы официантом,"Мы ищем активного, жизнерадостного и дружелюбн..."
4,Официант,"Рестораны, где работал","Мы ищем активного, жизнерадостного и дружелюбн..."
...,...,...,...
80504,Кладовщик,Опыт работы с программами,"ОАО ""Октябрьский электровагоноремонтный завод""..."
80505,Кладовщик,Опыт с инструментарием,"ОАО ""Октябрьский электровагоноремонтный завод""..."
80506,Кладовщик,Типы работ,"ОАО ""Октябрьский электровагоноремонтный завод""..."
80507,Кладовщик,Сколько времени вы потратили на поиск работы к...,"ОАО ""Октябрьский электровагоноремонтный завод""..."


In [46]:
train_df.sample(1)['text'].values[0]

'Обязанности:  Организация проведения отпуска товарно-материальных ценностей, осуществление подготовки товара к отгрузке (проверка факта оплаты товара, оформление необходимой документации, проверка целостности комплектации и товарного вида, демонстрация клиенту, доставка товара до автомобиля клиента). Проверка состояния, внешнего вида (целостности упаковки); Обеспечение надежной упаковки товара, предназначенного к отгрузке с сопровождением копиями отгрузочных документов; Участие в погрузо-разгрузочных работах; Участие в проведении инвентаризаций товарно-материальных ценностей.  Требования:  Опыт работы на аналогичной должности; Пользователь ПК: Word, Excel; Внимательность, ответственность.  Условия:  График работы: 5/2, 9-19, 10-20; Испытательный срок-1 месяц; Трудоустройство по ТК РФ, соц.пакет; Место работы ст.м. Электросила; Возможность карьерного роста; Молодой, дружный коллектив; Полный рабочий день, на территории работодателя. '

In [81]:
def generate_target(position, question, text):
    question_words = preprocess_text(question, 2)
    print(question_words)
    target = []
    for word in question_words:
        start = text.find(word) + len(word)
        i = start
        print(i)
        while not text[i] in punctuation and i < len(text):
            i += 1
        target.append(text[start:i])
        i = start
        while not text[i] in punctuation and i > 0:
            i -= 1
        target.append(text[start-1:i])
    return target

In [82]:
generate_target(train_data[0]['position'], train_data[0]['question'], train_data[0]['text'])

['профессиональный образование', 'сфера']
27
4


['остного и дружелюбного официанта', '', 'щем активного', '']

In [51]:
import pickle
class BagOfWordCoder:
    vectorizer = None
    def __init__(self, path_to_corpus):
        if not path_to_corpus.endswith(".vec"):
            raise Exception(f"Need to be *.vec file")
        with open(path_to_corpus, 'rb') as f:
            vectorizer = pickle.load(f)
    
    def vectorize(self, text):
        return vectorizer.transform(text)
    
    def get_text(self, matrix):
        return le.inverse_transform(matrix)
    

In [53]:
vectoriser = BagOfWordCoder('vectorizer.vec')